## Expansion City Factors

##### by: Jonathan Sax
##### Sports Management Analytics - Spring 2023

##### <b> Background Information </b>
##### For our final group presentation, we were asked to evaluate the optimal location for an expansion team of a professional sports league, and provide reasoning for this decision based on our learnings from the Sports Management Analytics curriculum. Eric and I relied upon metropolitan statistical area (MSA) data to choose where the NFL's 33rd team should reside.

In [104]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
import matplotlib.pylab as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [105]:
X = pd.read_excel("Expansion City.xlsx")

In [106]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Candidate City          10 non-null     object 
 1   Metro Pop '21           10 non-null     int64  
 2   City Pop '21            10 non-null     int64  
 3   % Growth (11y)          10 non-null     float64
 4   Distance                10 non-null     float64
 5   Income per Capita       10 non-null     int64  
 6   Fortune 500             10 non-null     int64  
 7   Other Teams             10 non-null     int64  
 8   Unemployment Rate Rank  10 non-null     int64  
 9   Remote Work Rank        10 non-null     int64  
 10  Public Transit Rank     10 non-null     int64  
 11  City Pop (2010 CENSUS)  10 non-null     int64  
 12  Unnamed: 12             0 non-null      float64
 13  Unnamed: 13             0 non-null      float64
 14  Unnamed: 14             0 non-null      float

In [107]:
X = X.dropna(axis=1, how='all')
X

,Candidate City,Metro Pop '21,City Pop '21,% Growth (11y),Distance,Income per Capita,Fortune 500,Other Teams,Unemployment Rate Rank,Remote Work Rank,Public Transit Rank,City Pop (2010 CENSUS)
0,Chicago,9509934,2696555,0.035502,0.0,41821,35,4,301,54,22,2695598
1,San Antonio,2601788,1451853,9.375120,196.9,28579,2,1,250,16,41,1327407
2,Oklahoma City,1441647,687725,18.573480,299.0,33162,2,1,78,38,94,579999
3,St. Louis,2216000,293310,-8.137954,243.0,45307,8,2,91,34,91,319294
4,Sacramento,2155000,525041,12.551877,95.4,37638,0,1,308,106,70,466488
5,Salt Lake City,1180000,200478,7.529500,515.0,42081,0,2,51,30,101,186440
6,San Diego,3272000,1381611,5.676066,118.0,46460,2,1,228,142,19,1307402
7,Oakland,4623264,433823,11.030548,5.0,49970,0,1,187,95,13,390724
8,San Jose,1799000,983489,3.969271,48.0,53574,8,2,174,121,15,945942
9,Portland,2510000,641162,9.830140,174.0,47289,2,2,212,20,10,583776


### Now we will give each column input equal weight into the final score, and normalize each column input. The resulting output shows that all the values are in the range 0 to 1.

In [108]:
# Drop City column from the front of the dataframe 
# so SKlearn can evaluate the whole df correctly

X2 = X.drop(['Candidate City'], axis=1)

In [109]:
X2

,Metro Pop '21,City Pop '21,% Growth (11y),Distance,Income per Capita,Fortune 500,Other Teams,Unemployment Rate Rank,Remote Work Rank,Public Transit Rank,City Pop (2010 CENSUS)
0,9509934,2696555,0.035502,0.0,41821,35,4,301,54,22,2695598
1,2601788,1451853,9.375120,196.9,28579,2,1,250,16,41,1327407
2,1441647,687725,18.573480,299.0,33162,2,1,78,38,94,579999
3,2216000,293310,-8.137954,243.0,45307,8,2,91,34,91,319294
4,2155000,525041,12.551877,95.4,37638,0,1,308,106,70,466488
5,1180000,200478,7.529500,515.0,42081,0,2,51,30,101,186440
6,3272000,1381611,5.676066,118.0,46460,2,1,228,142,19,1307402
7,4623264,433823,11.030548,5.0,49970,0,1,187,95,13,390724
8,1799000,983489,3.969271,48.0,53574,8,2,174,121,15,945942
9,2510000,641162,9.830140,174.0,47289,2,2,212,20,10,583776


In [110]:
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(X2)
normalized_df = pd.DataFrame(normalized_data, columns=X2.columns)

In [111]:
normalized_df

,Metro Pop '21,City Pop '21,% Growth (11y),Distance,Income per Capita,Fortune 500,Other Teams,Unemployment Rate Rank,Remote Work Rank,Public Transit Rank,City Pop (2010 CENSUS)
0,1.000000,1.000000,0.305991,0.000000,0.529786,1.000000,1.000000,0.972763,0.301587,0.131868,1.000000
1,0.170684,0.501337,0.655640,0.382330,0.000000,0.057143,0.000000,0.774319,0.000000,0.340659,0.454721
2,0.031410,0.195205,1.000000,0.580583,0.183357,0.057143,0.000000,0.105058,0.174603,0.923077,0.156849
3,0.124371,0.037191,0.000000,0.471845,0.669254,0.228571,0.333333,0.155642,0.142857,0.890110,0.052948
4,0.117048,0.130029,0.774568,0.185243,0.362432,0.000000,0.000000,1.000000,0.714286,0.659341,0.111610
5,0.000000,0.000000,0.586545,1.000000,0.540188,0.000000,0.333333,0.000000,0.111111,1.000000,0.000000
6,0.251142,0.473196,0.517158,0.229126,0.715383,0.057143,0.000000,0.688716,1.000000,0.098901,0.446748
7,0.413360,0.093485,0.717614,0.009709,0.855811,0.000000,0.000000,0.529183,0.626984,0.032967,0.081415
8,0.074310,0.313697,0.453260,0.093204,1.000000,0.228571,0.333333,0.478599,0.833333,0.054945,0.302692
9,0.159665,0.176551,0.672674,0.337864,0.748550,0.057143,0.333333,0.626459,0.031746,0.000000,0.158354


In [112]:
# Re-add City column to the front of the dataframe now that columns are normalized

City = X['Candidate City'].to_numpy()

In [113]:
normalized_df.insert(0, 'Candidate City', City)

In [114]:
normalized_df

,Candidate City,Metro Pop '21,City Pop '21,% Growth (11y),Distance,Income per Capita,Fortune 500,Other Teams,Unemployment Rate Rank,Remote Work Rank,Public Transit Rank,City Pop (2010 CENSUS)
0,Chicago,1.000000,1.000000,0.305991,0.000000,0.529786,1.000000,1.000000,0.972763,0.301587,0.131868,1.000000
1,San Antonio,0.170684,0.501337,0.655640,0.382330,0.000000,0.057143,0.000000,0.774319,0.000000,0.340659,0.454721
2,Oklahoma City,0.031410,0.195205,1.000000,0.580583,0.183357,0.057143,0.000000,0.105058,0.174603,0.923077,0.156849
3,St. Louis,0.124371,0.037191,0.000000,0.471845,0.669254,0.228571,0.333333,0.155642,0.142857,0.890110,0.052948
4,Sacramento,0.117048,0.130029,0.774568,0.185243,0.362432,0.000000,0.000000,1.000000,0.714286,0.659341,0.111610
5,Salt Lake City,0.000000,0.000000,0.586545,1.000000,0.540188,0.000000,0.333333,0.000000,0.111111,1.000000,0.000000
6,San Diego,0.251142,0.473196,0.517158,0.229126,0.715383,0.057143,0.000000,0.688716,1.000000,0.098901,0.446748
7,Oakland,0.413360,0.093485,0.717614,0.009709,0.855811,0.000000,0.000000,0.529183,0.626984,0.032967,0.081415
8,San Jose,0.074310,0.313697,0.453260,0.093204,1.000000,0.228571,0.333333,0.478599,0.833333,0.054945,0.302692
9,Portland,0.159665,0.176551,0.672674,0.337864,0.748550,0.057143,0.333333,0.626459,0.031746,0.000000,0.158354


### Since six of the input columns have a positive correlation between expansion city attractiveness and the input, we use the normalized input values when calculating our final score average. 

### Since four of the input columns have a negative correlation between expansion city attractiveness and the input, we invert those normalized input values when calculating the final score average.

In [118]:
# Specify the columns to be added and inverted

columns_to_add = ["Metro Pop '21" , "City Pop '21", "% Growth (11y)", 'Distance', 'Income per Capita', 
                 'Fortune 500']
columns_to_invert = ['Other Teams', 'Unemployment Rate Rank', 'Remote Work Rank', 'Public Transit Rank']


In [120]:
# Inverting the values of the selected columns
normalized_df[columns_to_invert] = 1 - normalized_df[columns_to_invert]

normalized_df

,Candidate City,Metro Pop '21,City Pop '21,% Growth (11y),Distance,Income per Capita,Fortune 500,Other Teams,Unemployment Rate Rank,Remote Work Rank,Public Transit Rank,City Pop (2010 CENSUS)
0,Chicago,1.000000,1.000000,0.305991,0.000000,0.529786,1.000000,0.000000,0.027237,0.698413,0.868132,1.000000
1,San Antonio,0.170684,0.501337,0.655640,0.382330,0.000000,0.057143,1.000000,0.225681,1.000000,0.659341,0.454721
2,Oklahoma City,0.031410,0.195205,1.000000,0.580583,0.183357,0.057143,1.000000,0.894942,0.825397,0.076923,0.156849
3,St. Louis,0.124371,0.037191,0.000000,0.471845,0.669254,0.228571,0.666667,0.844358,0.857143,0.109890,0.052948
4,Sacramento,0.117048,0.130029,0.774568,0.185243,0.362432,0.000000,1.000000,0.000000,0.285714,0.340659,0.111610
5,Salt Lake City,0.000000,0.000000,0.586545,1.000000,0.540188,0.000000,0.666667,1.000000,0.888889,0.000000,0.000000
6,San Diego,0.251142,0.473196,0.517158,0.229126,0.715383,0.057143,1.000000,0.311284,0.000000,0.901099,0.446748
7,Oakland,0.413360,0.093485,0.717614,0.009709,0.855811,0.000000,1.000000,0.470817,0.373016,0.967033,0.081415
8,San Jose,0.074310,0.313697,0.453260,0.093204,1.000000,0.228571,0.666667,0.521401,0.166667,0.945055,0.302692
9,Portland,0.159665,0.176551,0.672674,0.337864,0.748550,0.057143,0.666667,0.373541,0.968254,1.000000,0.158354


In [121]:
# Calculate the final score and add final score to the end of the dataframe

normalized_df['Final Score'] = (normalized_df[columns_to_add].sum(axis=1) + normalized_df[columns_to_invert].sum(axis=1)) / 10
normalized_df

,Candidate City,Metro Pop '21,City Pop '21,% Growth (11y),Distance,Income per Capita,Fortune 500,Other Teams,Unemployment Rate Rank,Remote Work Rank,Public Transit Rank,City Pop (2010 CENSUS),Final Score
0,Chicago,1.000000,1.000000,0.305991,0.000000,0.529786,1.000000,0.000000,0.027237,0.698413,0.868132,1.000000,0.542956
1,San Antonio,0.170684,0.501337,0.655640,0.382330,0.000000,0.057143,1.000000,0.225681,1.000000,0.659341,0.454721,0.465216
2,Oklahoma City,0.031410,0.195205,1.000000,0.580583,0.183357,0.057143,1.000000,0.894942,0.825397,0.076923,0.156849,0.484496
3,St. Louis,0.124371,0.037191,0.000000,0.471845,0.669254,0.228571,0.666667,0.844358,0.857143,0.109890,0.052948,0.400929
4,Sacramento,0.117048,0.130029,0.774568,0.185243,0.362432,0.000000,1.000000,0.000000,0.285714,0.340659,0.111610,0.319569
5,Salt Lake City,0.000000,0.000000,0.586545,1.000000,0.540188,0.000000,0.666667,1.000000,0.888889,0.000000,0.000000,0.468229
6,San Diego,0.251142,0.473196,0.517158,0.229126,0.715383,0.057143,1.000000,0.311284,0.000000,0.901099,0.446748,0.445553
7,Oakland,0.413360,0.093485,0.717614,0.009709,0.855811,0.000000,1.000000,0.470817,0.373016,0.967033,0.081415,0.490084
8,San Jose,0.074310,0.313697,0.453260,0.093204,1.000000,0.228571,0.666667,0.521401,0.166667,0.945055,0.302692,0.446283
9,Portland,0.159665,0.176551,0.672674,0.337864,0.748550,0.057143,0.666667,0.373541,0.968254,1.000000,0.158354,0.516091


In [122]:
df_rankings = normalized_df.sort_values(by='Final Score', ascending=False)
df_rankings

,Candidate City,Metro Pop '21,City Pop '21,% Growth (11y),Distance,Income per Capita,Fortune 500,Other Teams,Unemployment Rate Rank,Remote Work Rank,Public Transit Rank,City Pop (2010 CENSUS),Final Score
0,Chicago,1.000000,1.000000,0.305991,0.000000,0.529786,1.000000,0.000000,0.027237,0.698413,0.868132,1.000000,0.542956
9,Portland,0.159665,0.176551,0.672674,0.337864,0.748550,0.057143,0.666667,0.373541,0.968254,1.000000,0.158354,0.516091
7,Oakland,0.413360,0.093485,0.717614,0.009709,0.855811,0.000000,1.000000,0.470817,0.373016,0.967033,0.081415,0.490084
2,Oklahoma City,0.031410,0.195205,1.000000,0.580583,0.183357,0.057143,1.000000,0.894942,0.825397,0.076923,0.156849,0.484496
5,Salt Lake City,0.000000,0.000000,0.586545,1.000000,0.540188,0.000000,0.666667,1.000000,0.888889,0.000000,0.000000,0.468229
1,San Antonio,0.170684,0.501337,0.655640,0.382330,0.000000,0.057143,1.000000,0.225681,1.000000,0.659341,0.454721,0.465216
8,San Jose,0.074310,0.313697,0.453260,0.093204,1.000000,0.228571,0.666667,0.521401,0.166667,0.945055,0.302692,0.446283
6,San Diego,0.251142,0.473196,0.517158,0.229126,0.715383,0.057143,1.000000,0.311284,0.000000,0.901099,0.446748,0.445553
3,St. Louis,0.124371,0.037191,0.000000,0.471845,0.669254,0.228571,0.666667,0.844358,0.857143,0.109890,0.052948,0.400929
4,Sacramento,0.117048,0.130029,0.774568,0.185243,0.362432,0.000000,1.000000,0.000000,0.285714,0.340659,0.111610,0.319569


In [123]:
final_rankings = ['Candidate City', 'Final Score']
new_df = pd.DataFrame(df_rankings, columns=final_rankings)

In [124]:
# Removing all of the inputs and creating a new dataframe with just the final score

new_df

,Candidate City,Final Score
0,Chicago,0.542956
9,Portland,0.516091
7,Oakland,0.490084
2,Oklahoma City,0.484496
5,Salt Lake City,0.468229
1,San Antonio,0.465216
8,San Jose,0.446283
6,San Diego,0.445553
3,St. Louis,0.400929
4,Sacramento,0.319569


In [125]:
# Adding rank and background gradient

new_df['Rank'] = new_df['Final Score'].rank(ascending=False)
new_df['Rank'] = new_df['Rank'].astype(str).apply(lambda x: x.replace('.0',''))
first_column = new_df.pop('Rank')
new_df.insert(0, 'Rank', first_column)

final = new_df.sort_values(by='Final Score', ascending = False)
rounder = {'Final Score':'{:,.2f}'}
final.style.format(rounder).background_gradient(cmap= 'Blues')

,Rank,Candidate City,Final Score
0,1,Chicago,0.54
9,2,Portland,0.52
7,3,Oakland,0.49
2,4,Oklahoma City,0.48
5,5,Salt Lake City,0.47
1,6,San Antonio,0.47
8,7,San Jose,0.45
6,8,San Diego,0.45
3,9,St. Louis,0.40
4,10,Sacramento,0.32
